# Senators notebook

Parses datasets/senado/senators_with_memberships-**termid**.json creating / updating the following polare entites:
* Agents
* Memberships (Senate, Parties)
* Organizations (Senate)
* Posts (Senate)
* Roles (Afiliate, Senator)

## 1. Imports and Constants Declaration

In [10]:
import json   # reads scrapped data
import pandas as pd # helps with file management and visualization
from uuid import uuid4 # creates new uuids for Posts

termid = 55 # legislatura

PROV = 'sen'
FILE_SUFFIX = termid
SUB_DIR = '{:}/'.format(termid)
JSON_PATH =  'datasets/senado/'

BASE_PATH = 'datasets/slp/'
AGENTS_PATH = 'datasets/slp/agents/'
MEMBERSHIPS_PATH = 'datasets/slp/memberships/'
ORGANIZATIONS_PATH = 'datasets/slp/organizations/'
ROLES_PATH = 'datasets/slp/roles/'
POSTS_PATH = 'datasets/slp/posts/'

SENATE_URI = '81311052-e5b6-46fe-87ba-83865fa0ffb0' 
SENATOR_URI = 'd57a29ff-c69a-4b32-b98a-3dd8f204c0a3' 
AFFILIATE_URI = '6a688541-b16a-45ca-8aa9-fa700373279f' 


 ## 2. Getting the Parties

In [11]:
filename = 'organizations.csv'
file_path = '{:}{:}'.format(BASE_PATH, filename)
df_parties = pd.read_csv(file_path, sep= ';', encoding= 'utf-8', index_col=None)
df_parties.set_index('sigla', inplace=True)


parties_d = df_parties['resource_uri'].to_dict()                        
print({k:parties_d[k]
    for k in ['PT', 'PMDB', 'DEM']})

{'PT': '7b2138a6-df61-11e7-baf1-c82a144c0a85', 'PMDB': '7b210cfa-df61-11e7-9864-c82a144c0a85', 'DEM': '7b20ed7e-df61-11e7-bb65-c82a144c0a85'}


## 3. Reads scrapped json

In [12]:
filename = 'senator_with_memberships-{:}.json'.format(FILE_SUFFIX) 
file_path = '{:}{:}'.format(JSON_PATH, filename)
with open(file_path, mode='r') as f:
    senatorsstr = f.read()
f.close()

senators_with_memberships = json.loads(senatorsstr)

## 4. Agents

In [13]:
mapping = {'skos:prefLabel': 'sen:CodigoParlamentar',
  'foaf:name':'sen:NomeCompletoParlamentar',
  'rdfs:label':'sen:NomeParlamentar',
  'resource_uri':'slp:resource_uri'           
}

agents = [] 
for swm_d in senators_with_memberships:
    agent_d = {mapping[k]: swm_d.get(k, None) for k in mapping}
    if agent_d:
        agents.append(agent_d)

filename = 'senators-{:}.csv'.format(PROV) 
file_path = '{:}{:}{:}'.format(AGENTS_PATH, SUB_DIR, filename)
df = pd.DataFrame.from_dict(agents)    
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)
df.head()

,sen:CodigoParlamentar,sen:NomeCompletoParlamentar,sen:NomeParlamentar,slp:resource_uri
0,715,Armando de Queiroz Monteiro Neto,Armando Monteiro,21472871-dd36-4d0c-874e-d442e3f6eb6a
1,846,Aloysio Nunes Ferreira Filho,Aloysio Nunes Ferreira,6b45dd59-d18c-4cf7-85f4-385b8b7400d6
2,3,Antonio Carlos Valadares,Antonio Carlos Valadares,1065859f-e76a-4bad-9c44-e404e24adc54
3,4988,Ana Amélia de Lemos,Ana Amélia,14a3ea0b-56ba-441d-a6f1-f69a29bf3ec3
4,5164,Ataídes de Oliveira,Ataídes Oliveira,d2fd7a9e-eb01-4c1d-a326-8fbbbf5a9164


 ## 5. Senate Org
 
 ### 5.1 Membership Senator
 #### 5.1.1 Process

In [18]:
mapping = {
    'code': 'sen:CodigoMandato',
    'area': 'sen:UfParlamentar',
    'startDate': 'sen:startDate', 
    'finishDate': 'sen:finishDate',
    'resource_uri':'slp:resource_uri'
}

memberships = [] 
for swm_d in senators_with_memberships:
    member_uri = swm_d['resource_uri'] 
    terms = swm_d['terms']
    if terms:
        for term_d in terms:
            memberships_d = {mapping[k]: term_d.get(k, None) for k in mapping}
            if memberships_d:
                memberships_d['org:member'] = member_uri
                memberships_d['org:role'] = SENATOR_URI
                memberships.append(memberships_d)



df = pd.DataFrame.from_dict(memberships)    
df.head()            

,org:member,org:role,sen:CodigoMandato,sen:UfParlamentar,sen:finishDate,sen:startDate,slp:resource_uri
0,21472871-dd36-4d0c-874e-d442e3f6eb6a,d57a29ff-c69a-4b32-b98a-3dd8f204c0a3,443,PE,None,2016-05-09,e9932b05-852a-446e-a4ce-51ab18351d30
1,21472871-dd36-4d0c-874e-d442e3f6eb6a,d57a29ff-c69a-4b32-b98a-3dd8f204c0a3,443,PE,2015-01-01,2014-11-15,0ab6250c-1305-411e-ad98-a1c2abba98f8
2,21472871-dd36-4d0c-874e-d442e3f6eb6a,d57a29ff-c69a-4b32-b98a-3dd8f204c0a3,443,PE,2014-07-16,2011-02-01,b1e007b9-6fe2-4440-abab-56526952585d
3,6b45dd59-d18c-4cf7-85f4-385b8b7400d6,d57a29ff-c69a-4b32-b98a-3dd8f204c0a3,485,SP,2017-03-06,2011-02-01,fcb59e3d-054d-49b6-a0e8-fe51b6949777
4,1065859f-e76a-4bad-9c44-e404e24adc54,d57a29ff-c69a-4b32-b98a-3dd8f204c0a3,474,SE,None,2018-03-23,8018cc6a-d48b-4d0e-a1e5-24c8c4d467f2


 #### 5.1.2 Save

In [19]:
filename = 'memberships_with_senate-{:}.csv'.format(PROV, FILE_SUFFIX) 
file_path = '{:}{:}{:}'.format(MEMBERSHIPS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

### 5.3 Posts

#### 5.3.1 Posts Process

In [20]:
df = df.rename({'slp:resource_uri':'org:role'}, axis='columns')
df['resource_uri'] = [str(uuid4()) for _ in range(df.shape[0])]
df['org:postIn'] = [SENATE_URI]*df.shape[0]
df.drop('rdfs:label', axis=1, inplace=True)
df.head()

,org:member,org:role,sen:CodigoMandato,sen:UfParlamentar,sen:finishDate,sen:startDate,org:role,resource_uri,org:postIn
0,21472871-dd36-4d0c-874e-d442e3f6eb6a,d57a29ff-c69a-4b32-b98a-3dd8f204c0a3,443,PE,None,2016-05-09,e9932b05-852a-446e-a4ce-51ab18351d30,7f91b54d-2914-4870-af1e-20bb29d863c3,81311052-e5b6-46fe-87ba-83865fa0ffb0
1,21472871-dd36-4d0c-874e-d442e3f6eb6a,d57a29ff-c69a-4b32-b98a-3dd8f204c0a3,443,PE,2015-01-01,2014-11-15,0ab6250c-1305-411e-ad98-a1c2abba98f8,42f5f3ad-a1ad-4062-a5cc-457fb5041c8b,81311052-e5b6-46fe-87ba-83865fa0ffb0
2,21472871-dd36-4d0c-874e-d442e3f6eb6a,d57a29ff-c69a-4b32-b98a-3dd8f204c0a3,443,PE,2014-07-16,2011-02-01,b1e007b9-6fe2-4440-abab-56526952585d,d3d64618-3031-46b1-b483-f5fc08b22940,81311052-e5b6-46fe-87ba-83865fa0ffb0
3,6b45dd59-d18c-4cf7-85f4-385b8b7400d6,d57a29ff-c69a-4b32-b98a-3dd8f204c0a3,485,SP,2017-03-06,2011-02-01,fcb59e3d-054d-49b6-a0e8-fe51b6949777,a0eed1ed-669a-4de1-80e4-3c7f075a6b02,81311052-e5b6-46fe-87ba-83865fa0ffb0
4,1065859f-e76a-4bad-9c44-e404e24adc54,d57a29ff-c69a-4b32-b98a-3dd8f204c0a3,474,SE,None,2018-03-23,8018cc6a-d48b-4d0e-a1e5-24c8c4d467f2,13922ec1-f097-4b98-b5fd-03af5b25858c,81311052-e5b6-46fe-87ba-83865fa0ffb0


In [21]:
df.shape

(179, 9)

#### 5.3.3 Posts save

In [14]:
filename = 'senator_posts-{:}.csv'.format(PROV, FILE_SUFFIX) 
file_path = '{:}{:}{:}'.format(POSTS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

 ## 6. Parties Org

 ### 6.1 Membership
 
 #### 6.1.1 Process

In [10]:
mapping = {
    'sigla': 'sen:SiglaPartido',
    'startDate': 'sen:DataFiliacao' ,
    'finishDate': 'sen:DataDesfiliacao',
    'resource_uri': 'slp:resource_uri'
}

memberships = [] 
for swm_d in senators_with_memberships:
    affiliations = swm_d['affiliations']
    if affiliations:
        for affiliation_d in affiliations:
            if affiliation_d['sigla'] in parties_d:
                memberships_d = {mapping[k]: affiliation_d.get(k, None) for k in mapping}
                if memberships_d:
                    memberships_d['org:member'] = member_uri
                    memberships_d['org:role'] = AFFILIATE_URI
                    memberships.append(memberships_d)



                    
df = pd.DataFrame.from_dict(memberships)    
parties = df['sen:SiglaPartido'].values
df.head(10)            

,org:member,org:role,sen:DataDesfiliacao,sen:DataFiliacao,sen:SiglaPartido,slp:resource_uri
0,85e8b745-786b-4782-a8a1-e3b3164a3f22,6a688541-b16a-45ca-8aa9-fa700373279f,None,2015-02-01,PDT,0577ae6b-9fa4-4ff0-ac19-9dd5cd3598d1
1,85e8b745-786b-4782-a8a1-e3b3164a3f22,6a688541-b16a-45ca-8aa9-fa700373279f,None,2004-02-14,PP,b2ec603a-26b8-4e59-8a59-624d2f0973ac
2,85e8b745-786b-4782-a8a1-e3b3164a3f22,6a688541-b16a-45ca-8aa9-fa700373279f,2004-02-13,1999-02-01,PFL,3cf1400f-9fe7-4552-99ac-d0a4a52a1510
3,85e8b745-786b-4782-a8a1-e3b3164a3f22,6a688541-b16a-45ca-8aa9-fa700373279f,None,2011-02-03,PR,da9f5ea7-2636-4240-af94-94e0df597ae9
4,85e8b745-786b-4782-a8a1-e3b3164a3f22,6a688541-b16a-45ca-8aa9-fa700373279f,None,2011-02-03,DEM,89a85f62-3dce-499c-a72a-0917f5bf5240
5,85e8b745-786b-4782-a8a1-e3b3164a3f22,6a688541-b16a-45ca-8aa9-fa700373279f,None,2015-02-01,PDT,5d602564-ea0f-4b99-86fe-7ed822778f85
6,85e8b745-786b-4782-a8a1-e3b3164a3f22,6a688541-b16a-45ca-8aa9-fa700373279f,None,1999-02-01,PSDB,70e5af79-d369-4e74-b4ad-4decab26ef97
7,85e8b745-786b-4782-a8a1-e3b3164a3f22,6a688541-b16a-45ca-8aa9-fa700373279f,None,2015-02-01,PV,27187447-24a0-4e48-900d-5207e5ce37f5
8,85e8b745-786b-4782-a8a1-e3b3164a3f22,6a688541-b16a-45ca-8aa9-fa700373279f,None,2015-02-01,PMDB,3bd6d232-08e8-4654-acdb-a46a77de65ba
9,85e8b745-786b-4782-a8a1-e3b3164a3f22,6a688541-b16a-45ca-8aa9-fa700373279f,None,2011-01-27,PR,5401befb-466d-4949-890c-5969d74233e2


In [15]:
print(senators_with_memberships[0])

{'skos:prefLabel': '5573', 'rdfs:label': 'Abel Rebouças', 'foaf:name': 'Abel Rebouças São José', 'agent_resource_uri': '828a0e78-b457-47de-8112-a7cce8d24563', 'terms': [{'legislatura': '55', 'startDate': '2015-02-01', 'finishDate': '2019-01-31', 'skos:prefLabel': '492', 'natureza': 'BA', 'membership_resource_uri': '61e28c9c-c93b-40fc-a73d-3a963632df24', 'role_resource_uri': 'd69fcc38-21a6-41ee-8c0f-90be24aff805'}, {'legislatura': '56', 'startDate': '2019-02-01', 'finishDate': '2023-01-31', 'skos:prefLabel': '492', 'natureza': 'BA', 'membership_resource_uri': '669716c0-cf09-4d77-896b-8497c3943b34', 'role_resource_uri': 'd69fcc38-21a6-41ee-8c0f-90be24aff805'}], 'affiliations': [{'sigla': 'PDT', 'startDate': '2015-02-01', 'resource_uri': '0577ae6b-9fa4-4ff0-ac19-9dd5cd3598d1', 'role_resource_uri': 'ebf7f8f0-1b0a-4662-9ed8-c9bb19936ec0'}]}


 #### 6.1.2 Save

In [25]:
filename = 'memberships_at_parties-{:}.csv'.format(PROV) 
file_path = '{:}{:}{:}'.format(MEMBERSHIPS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

  ### 6.2 Roles at Parties
  #### 6.2.1 Process

In [20]:
df = df['org:role'].to_frame()
df = df.rename({'org:role':'slp:resource_uri'}, axis='columns')
df['rdfs:label'] = 'Affiliate'
df.head()

,slp:resource_uri,rdfs:label
0,70f12084-b63a-4bca-b140-e60480ef3802,Affiliate
1,1706c489-db73-4b74-a410-5e441c1e1cdd,Affiliate
2,ea9a2925-a286-4b13-b719-a030418dd8c5,Affiliate
3,11dccd25-3d27-4e70-af1d-bb0907675b2c,Affiliate
4,a2d74722-5042-4795-b72e-c4d3bba42fc9,Affiliate


 #### 6.2.2 Save

In [21]:
filename = 'roles_affiliatiate-{:}.csv'.format(PROV) 
file_path = '{:}{:}{:}'.format(ROLES_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

 ### 6.3 Posts at Parties
 #### 6.3.1 Process

In [22]:
df = df.rename({'slp:resource_uri':'org:role'}, axis='columns')
df['slp:resource_uri'] = [parties_d.get(sigla, None) for sigla in parties]
df.drop('rdfs:label', axis=1, inplace=True)
df.head()

,org:role,slp:resource_uri
0,70f12084-b63a-4bca-b140-e60480ef3802,7b210070-df61-11e7-97e9-c82a144c0a85
1,1706c489-db73-4b74-a410-5e441c1e1cdd,7b21152e-df61-11e7-bdcf-c82a144c0a85
2,ea9a2925-a286-4b13-b719-a030418dd8c5,7b21046c-df61-11e7-8094-c82a144c0a85
3,11dccd25-3d27-4e70-af1d-bb0907675b2c,7b211a38-df61-11e7-b1b8-c82a144c0a85
4,a2d74722-5042-4795-b72e-c4d3bba42fc9,7b20ed7e-df61-11e7-bb65-c82a144c0a85


 #### 6.3.2 Save

In [23]:
filename = 'posts_at_parties-{:}.csv'.format(PROV) 
file_path = '{:}{:}{:}'.format(POSTS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)